# Hyperparameter

Hyperparameter adalah parameter yag dapat disesuaikan untuk mengontrol proses pemgoptimalan model. Hyperparameter pada saat proses iterasi tidak akan diupdate atau sifatnya tetap.
- Epoch: proses berulang selama proses pelatihan, dimana 1 epoch = keseluruhan dataset telah dilatih.
- Batch jumlah sampel yang digunakan pada proses iterasi.
- Learning Rate: kecepatan model dalam belajar. Nilai learning rate yang kecil menyebabkan proses belajar model menjadi lambat dan sebaliknya jika learning rate besar, maka proses belajar model menjadi cepat.

# Loss Function

Digunakan untuk mengukur ketidakmiripan hasil dari prediksi model dengan nilai aktual.

In [1]:
X_actual = 10
X_pred = 9.6

loss = X_actual - X_pred
print(f"Loss: {loss}")

Loss: 0.40000000000000036


Pada contoh diatas selisih antara *X_actual* dan *X_pred* disebut *loss* atau *error*. Semakin besar kerugiannya (*loss*) maka performa model masih kurang baik dan sebaliknya jika semakin kecil maka performa model semakin baik.

Pada Pytroch terdapat beberapa fungsi loss, berikut tautannya https://pytorch.org/docs/stable/nn.html#loss-functions

In [5]:
import torch
from torch import nn

torch.manual_seed(42)
y_pred = torch.randn(size=(10, ))
y_actual = torch.randn(size=(10, ))

mse_loss = nn.MSELoss()
print(f"MSE Loss: {mse_loss(y_pred, y_actual).item()}")

MSE Loss: 1.1729463338851929


# Optimizer

- Digunakan untuk mengoptimalkan nilai-nilai dari parameter agar nilai kerugian (*loss*) semakin berkurang.
- Menggunakan Gradient Descent untuk melakukan optimasi parameter.
- Pada Pytroch htersedia beberapa optimizer yang dapat dilihat pada tautan https://pytorch.org/docs/stable/optim.html

In [3]:
from torch import nn, optim

# Build Model
model = nn.Sequential(
            # layer 1 (4 neurons) to layer 2 (2 neurons) with ReLU
            nn.Linear(4, 2),
            nn.ReLU(),
            
            # layer 2 (2 neurons) to layer 3 (1 neurons) with Sigmoid
            nn.Linear(2, 1),
            nn.Sigmoid(),
        )

optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

# Full Code

In [4]:
# Import modules
import torch

from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from torchmetrics.functional import r2_score

# Sample Dataset
torch.manual_seed(42)
X = torch.randn(size=(50, 4))
y = torch.randn(size=(50, 1))

# Tensor Dataset
tensor_dataset = TensorDataset(X, y)

# Dataloader
dataloader = DataLoader(tensor_dataset, batch_size=10, shuffle=True)

# Loss Function
mse_loss = nn.MSELoss()

# Optimizer
sgd_optimizer = optim.SGD(model.parameters(), lr=1)

# Loop Function
def loop_fn(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    loss_batches = []
    r2_score_batches = []
    
    for batch, (X, y) in enumerate(dataloader):
        # Forward Propagation
        pred = model(X)
        loss = loss_fn(pred, y)
        r_square = r2_score(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate loss each batch
        loss = loss.item()
        loss_batches.append(loss)
        
        # Calculate r-square each batch
        r_square = r_square.item()
        r2_score_batches.append(r_square)
        
        current_batch = batch * len(X)
        
        # Show Log
        print(f"batch {batch + 1}: [{current_batch:>3d}/{size:>3d}] | r2-score: {r_square:>8f} | loss: {loss:>7f}")
    print(f"R2 score  : {sum(r2_score_batches) / len(r2_score_batches)}")
    print(f"RMSE loss : {(sum(loss_batches) / len(loss_batches))**0.5}")
    
# Run Loop Function
epochs = 5

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    loop_fn(dataloader, model, mse_loss, sgd_optimizer)
    print("-" * 60, end="\n\n")
print("Done!")

Epoch 1
batch 1: [  0/ 50] | r2-score: -0.333630 | loss: 1.128540
batch 2: [ 10/ 50] | r2-score: -1.477357 | loss: 0.175021
batch 3: [ 20/ 50] | r2-score: -0.021762 | loss: 1.203532
batch 4: [ 30/ 50] | r2-score: -0.058999 | loss: 0.810576
batch 5: [ 40/ 50] | r2-score: -0.008366 | loss: 0.756777
R2 score  : -0.38002262115478513
RMSE loss : 0.9027120804519996
------------------------------------------------------------

Epoch 2
batch 1: [  0/ 50] | r2-score: -0.043790 | loss: 0.516787
batch 2: [ 10/ 50] | r2-score: -0.146816 | loss: 1.157540
batch 3: [ 20/ 50] | r2-score: -0.018677 | loss: 0.704841
batch 4: [ 30/ 50] | r2-score: -0.005922 | loss: 1.137529
batch 5: [ 40/ 50] | r2-score: -0.618832 | loss: 0.409459
R2 score  : -0.16680734157562255
RMSE loss : 0.8861327533078694
------------------------------------------------------------

Epoch 3
batch 1: [  0/ 50] | r2-score: 0.068780 | loss: 1.151492
batch 2: [ 10/ 50] | r2-score: -0.128021 | loss: 0.617208
batch 3: [ 20/ 50] | r2-score